In [1]:
import numpy as np
import cv2
import os

In [2]:
def dist(x1,x2):
    return np.sqrt(sum((x1-x2)**2))

In [3]:
def knn(X,Y,queryPoint,k=5):
    vals = []
    m = X.shape[0]
    
    for i in range(m):
        d = dist(queryPoint,X[i])
        vals.append((d,Y[i]))
        
    vals = sorted(vals)
    vals = vals[:k]
    
    vals = np.array(vals)
    
    new_vals = np.unique(vals[:,1],return_counts=True)
    index = new_vals[1].argmax()
    pred = new_vals[0][index]
    
    return pred

In [4]:
cap = cv2.VideoCapture(0)
face_cascade = cv2.CascadeClassifier('haarcascade_frontalface_alt.xml')

skip=0
dataset_path = './'

face_data=[]
labels =[]

class_id=0
names = {}

#Data Preparation
for fx in os.listdir(dataset_path):
    if(fx.endswith('.npy')):
        #Create a mapping btw class_id and name
        names[class_id] = fx[:-4]
        data_item = np.load(dataset_path+fx)
        face_data.append(data_item)
        
        #Create labels for the class
        target = class_id*np.ones((data_item.shape[0],))
        class_id += 1
        labels.append(target)

face_dataset = np.concatenate(face_data,axis=0)
face_labels = np.concatenate(labels,axis=0)

#Testing
ret,frame = cap.read()
faces = face_cascade.detectMultiScale(frame,1.3,5)
for (x,y,w,h) in faces:
    offset = 10
    face_section = frame[y-offset:y+h+offset,x-offset:x+w+offset]
    face_section = cv2.resize(face_section,(100,100))
    
    #Predicted Label
    out = knn(face_dataset,face_labels,face_section.flatten())
    
    #Display on the screen the name and rectangle around it
    pred_name = names[int(out)]
    cv2.putText(frame,pred_name,(x,y-10),cv2.FONT_HERSHEY_SIMPLEX,1,(255,0,0),2,cv2.LINE_AA)
    cv2.rectangle(frame,(x,y),(x+w,y+h),(255,0,0),2)

cv2.imshow("Faces",frame)

key = cv2.waitKey(1) & 0xFF